# Basic Neural Net from scratch
## with pytorch 
This notebook has the purpose of creating a simple neural net from scratch only wiht pytorch. 

To do this, I need: 
 - linear
 - Relu
 - sigmoid
 - softmax
 - backwards, i.e. the parameters, gradients
 - some kind of loss function, MSE, RMSE? - cross entropy loss it is
 

class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        movies = self.movie_factors[x[:,1]]
        res = (users*movies).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)



before doing the model - do not forget to clean the data and transform it so that most of the variation is between 0 and 1.

With e.g. divide it by its maximum, so that it becomes like a percentage or use a sigmoid.

## import data

In [1]:
import pandas as pd

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    df = pd.read_csv("/kaggle/input/btcusdt-2023-6-9/btcusdt-2023-6_9.csv", index_col=0).reset_index(drop=True)
else:
    df = pd.read_csv("lesson5-random-forests/btc-data/btcusdt-2023-6_9.csv", index_col=0).reset_index(drop=True)


print(df.shape)
df.head(3)

(11716, 6)


,time,open,high,low,close,vol
0,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675
1,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680
2,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080


In [2]:
# shift data 3 times, so that in one row there is information on the last 3 candles 
# therefore: the original candle data is the target data

df_s1 = df.shift(1).add_suffix("_s1")
df_s2 = df.shift(2).add_suffix("_s2")
df_s3 = df.shift(3).add_suffix("_s3")

print(df_s3.shape)
df_s3.head(3)

(11716, 6)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3
0,None,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,NaN


In [3]:
df_merge = pd.concat([df_s3,df_s2, df_s1, df], axis=1)
print(df_merge.shape)
df_merge.head(5)

(11716, 24)


,time_s3,open_s3,high_s3,low_s3,close_s3,vol_s3,time_s2,open_s2,high_s2,low_s2,...,high_s1,low_s1,close_s1,vol_s1,time,open,high,low,close,vol
0,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675
1,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,27108.1,27080.6,27096.9,386.675,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680
2,None,NaN,NaN,NaN,NaN,NaN,2023.06.01 00:00,27103.1,27108.1,27080.6,...,27096.9,27036.7,27047.0,408.680,2023.06.01 00:30,27047.0,27077.4,27041.0,27054.9,275.080
3,2023.06.01 00:00,27103.1,27108.1,27080.6,27096.9,386.675,2023.06.01 00:15,27096.9,27096.9,27036.7,...,27077.4,27041.0,27054.9,275.080,2023.06.01 00:45,27054.9,27084.0,27054.8,27084.0,218.143
4,2023.06.01 00:15,27096.9,27096.9,27036.7,27047.0,408.680,2023.06.01 00:30,27047.0,27077.4,27041.0,...,27084.0,27054.8,27084.0,218.143,2023.06.01 01:00,27084.0,27113.9,27073.5,27100.0,329.412


In [4]:
df_merge.columns

Index(['time_s3', 'open_s3', 'high_s3', 'low_s3', 'close_s3', 'vol_s3',
       'time_s2', 'open_s2', 'high_s2', 'low_s2', 'close_s2', 'vol_s2',
       'time_s1', 'open_s1', 'high_s1', 'low_s1', 'close_s1', 'vol_s1', 'time',
       'open', 'high', 'low', 'close', 'vol'],
      dtype='object')

In [5]:
# do not use the time columns 

df_train = df_merge.filter(items = ['open_s3', 'high_s3', 'low_s3', 'close_s3', 'vol_s3',
       'open_s2', 'high_s2', 'low_s2', 'close_s2', 'vol_s2',
       'open_s1', 'high_s1', 'low_s1', 'close_s1', 'vol_s1', 
       'open', 'high', 'low', 'close', 'vol']).dropna()
print(df_train.shape)
df_train.head(3)

(11713, 20)


,open_s3,high_s3,low_s3,close_s3,vol_s3,open_s2,high_s2,low_s2,close_s2,vol_s2,open_s1,high_s1,low_s1,close_s1,vol_s1,open,high,low,close,vol
3,27103.1,27108.1,27080.6,27096.9,386.675,27096.9,27096.9,27036.7,27047.0,408.680,27047.0,27077.4,27041.0,27054.9,275.080,27054.9,27084.0,27054.8,27084.0,218.143
4,27096.9,27096.9,27036.7,27047.0,408.680,27047.0,27077.4,27041.0,27054.9,275.080,27054.9,27084.0,27054.8,27084.0,218.143,27084.0,27113.9,27073.5,27100.0,329.412
5,27047.0,27077.4,27041.0,27054.9,275.080,27054.9,27084.0,27054.8,27084.0,218.143,27084.0,27113.9,27073.5,27100.0,329.412,27100.0,27159.0,27100.0,27142.4,979.655


In [6]:
#df_train
#df_train.filter(items = ["open_s3"])
#df_train.loc[["open_s3"]]



# manually calculate one iteration

In [7]:
import torch
from torch import tensor

## dep, indep & weights

In [8]:
t_dep = tensor(df_train['high'].values, dtype=torch.float)
print(t_dep.shape) # 1 D tensor
t_dep

torch.Size([11713])


tensor([27084.0000, 27113.9004, 27159.0000,  ..., 27058.1992, 27061.1992,
        27066.6992])

In [9]:
t_indep = tensor(df_train.loc[:, df_train.columns != "high"].values, dtype=torch.float)

t_indep.shape # 2 D tensor > rows with observations and columns with features

torch.Size([11713, 19])

The number of features needs to be the same as the lenght of the weights, because they will be multiplicated.

In [10]:
# coefficients = weights
# make them from -0.5 to 0.5 from a normal distribution
coeffs = torch.rand( t_indep.shape[1] )-0.5

print(coeffs.shape)
coeffs

torch.Size([19])


tensor([-0.0040, -0.0017, -0.2620,  0.0559,  0.2515,  0.2680,  0.0792, -0.1563,
        -0.4689,  0.4441, -0.2092, -0.4693, -0.0448, -0.4689, -0.4324,  0.1789,
        -0.3307, -0.4952, -0.4785])

In [11]:
(t_indep*coeffs)[0]

tensor([  -107.3060,    -45.9412,  -7094.3042,   1514.2396,     97.2566,
          7261.2510,   2146.5469,  -4224.6987, -12681.4277,    181.4939,
         -5658.3203, -12708.2090,  -1211.8651, -12685.8887,   -118.9354,
          4839.4443,  -8948.1279, -13412.4434,   -104.3844])

As you can see, there are a lot of numbers up and down, since the prices range roughly from 30k to about 60k. 

Note to myself: if it does not interfere with cross entropy then I should use log on all prices.
cross entropy uses log in its calculations and it depends on the implementation if it does interfere (I guess right now).

## making the first prediction

Since we are making a linear classifier the formula for yhat or the predictions is the sum of the products of coefficients and observations. 

In other words: 

$ y = x_0 \times b_0 + x_1 \times b_1 $

where $x_0$ is the first vector of observations (=features) and $b_0$ is the first coefficient.

and then is $y$ the vector with the predictions the same length as the data frame, so this formula can be read "rowwise".

In [12]:
preds = (t_indep*coeffs).sum(axis=1)
print(preds.shape) # 1d vector of predictions
preds[0]

torch.Size([11713])


tensor(-62961.6172)

In [13]:
# quick check for first row: 

print(t_indep[0])
print(coeffs)
print("\n first entry of Indep * coeffs: \n",t_indep[0][0]*coeffs[0])
print("\n Indep * coeffs: \n",t_indep[0]*coeffs)
print("\n sum of Indep * coeffs: \n", (t_indep[0]*coeffs).sum() ) #axis=1 is along the colums


tensor([27103.0996, 27108.0996, 27080.5996, 27096.9004,   386.6750, 27096.9004,
        27096.9004, 27036.6992, 27047.0000,   408.6800, 27047.0000, 27077.4004,
        27041.0000, 27054.9004,   275.0800, 27054.9004, 27054.8008, 27084.0000,
          218.1430])
tensor([-0.0040, -0.0017, -0.2620,  0.0559,  0.2515,  0.2680,  0.0792, -0.1563,
        -0.4689,  0.4441, -0.2092, -0.4693, -0.0448, -0.4689, -0.4324,  0.1789,
        -0.3307, -0.4952, -0.4785])

 first entry of Indep * coeffs: 
 tensor(-107.3060)

 Indep * coeffs: 
 tensor([  -107.3060,    -45.9412,  -7094.3042,   1514.2396,     97.2566,
          7261.2510,   2146.5469,  -4224.6987, -12681.4277,    181.4939,
         -5658.3203, -12708.2090,  -1211.8651, -12685.8887,   -118.9354,
          4839.4443,  -8948.1279, -13412.4434,   -104.3844])

 sum of Indep * coeffs: 
 tensor(-62961.6172)


## loss: cross entropy 

In [14]:
import torch.nn.functional as F
loss_ce = F.cross_entropy(preds, t_dep)
loss_mae = F.l1_loss(preds, t_dep)
loss_mae = F.mse_loss(preds, t_dep)

print("mean absolute error:",loss_mae)
print("mean squared error:",loss_mae)
print("cross entropy:",loss_ce)

mean absolute error: tensor(8.7773e+09)
mean squared error: tensor(8.7773e+09)
cross entropy: tensor(8.3253e+12)


## gradients & learning rate

In [15]:
coeffs

tensor([-0.0040, -0.0017, -0.2620,  0.0559,  0.2515,  0.2680,  0.0792, -0.1563,
        -0.4689,  0.4441, -0.2092, -0.4693, -0.0448, -0.4689, -0.4324,  0.1789,
        -0.3307, -0.4952, -0.4785])

In [16]:
coeffs.requires_grad_()

tensor([-0.0040, -0.0017, -0.2620,  0.0559,  0.2515,  0.2680,  0.0792, -0.1563,
        -0.4689,  0.4441, -0.2092, -0.4693, -0.0448, -0.4689, -0.4324,  0.1789,
        -0.3307, -0.4952, -0.4785], requires_grad=True)

In [17]:
# I am not sure if the above way works if I only use the loss functions in a functional style
# therefore, I document a way to use it instantiated as a class

from torch import nn

cel = nn.CrossEntropyLoss()

preds = (t_indep*coeffs.requires_grad_()).sum(axis=1)

loss = cel(preds,t_dep)
print(loss)
loss.backward(retain_graph=True)

gradients = coeffs.grad
print(gradients)

tensor(8.3253e+12, grad_fn=<DivBackward1>)
tensor([-5.5637e+11, -5.6059e+11, -6.9860e+11, -7.0251e+11,  5.3066e+12,
        -7.0252e+11, -6.8365e+11, -8.9695e+11, -7.2761e+11,  1.3745e+13,
        -7.2761e+11, -7.2464e+11, -7.7106e+11, -7.4167e+11,  1.5775e+12,
        -7.4168e+11, -7.6538e+11, -7.5986e+11,  3.9438e+11])


In [18]:
loss.backward(retain_graph=True)
print(coeffs.grad)

tensor([-1.1127e+12, -1.1212e+12, -1.3972e+12, -1.4050e+12,  1.0613e+13,
        -1.4050e+12, -1.3673e+12, -1.7939e+12, -1.4552e+12,  2.7490e+13,
        -1.4552e+12, -1.4493e+12, -1.5421e+12, -1.4833e+12,  3.1550e+12,
        -1.4834e+12, -1.5308e+12, -1.5197e+12,  7.8875e+11])


In [19]:
# initialise coeffs and define the independent variable 
t_indep*coeffs

# define it as a function 
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
    
preds = (t_indep*coeffs).sum(axis=1)

In [20]:
# define the loss function

# herer it is mean absolute error
loss = torch.abs(preds-t_dep).mean()

# define it in a function
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

# next steps

 - figure out when and how to use backward() and require_grad
 - put the pieces togehter: calc preds > calc loss > get gradients > subtract gradients*learningrate with preds

Note on gradients: \n
My current understand is that you need to specify the tensor of where to calculate the gradients on, to access them explicitly. \n
It seems, that the backward function needs to know the input, target and weights.

# og pytorch tutorial

https://pytorch.org/tutorials/beginner/nn_tutorial.html

relates directly to fast ai tutorial: 
https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch